In [1]:
import requests

In [2]:
from tqdm import tqdm

In [3]:
example = [[4.6,3.1,1.5,0.2]]

In [4]:
def get_predictions(examples, instance=requests, port=5000):
    payload = { "features": examples }
    return instance.post(f"http://127.0.0.1:{port}/invocations", json=payload)

In [5]:
def get_health(instance=requests, port=5000):
    instance.get(f"http://127.0.0.1:{port}/ping")

In [6]:
ports = {
    "plumber": 5001,
    "fastapi": 5000,
    "restrserve": 5002
}

In [7]:
many_examples = example*100

# New Requests

### Plumber

In [8]:
get_predictions(example*2,port=ports["plumber"]).json()

[0, 0]

In [9]:
for i in tqdm(range(1000)):
    _ = get_predictions(example,port=ports["plumber"])

100%|██████████| 1000/1000 [00:10<00:00, 95.28it/s]


In [10]:
for i in tqdm(range(1000)):
    _ = get_predictions(many_examples, port=ports["plumber"])

100%|██████████| 1000/1000 [00:16<00:00, 61.21it/s]


In [11]:
for i in tqdm(range(1000)):
    get_health(port=ports["plumber"])

100%|██████████| 1000/1000 [00:03<00:00, 313.47it/s]


### RestRserve

In [12]:
get_predictions(example*2,port=ports["restrserve"]).json()

{'outputs': [0, 0]}

In [13]:
for i in tqdm(range(1000)):
    _ = get_predictions(example,port=ports["restrserve"])

100%|██████████| 1000/1000 [00:32<00:00, 30.59it/s]


In [14]:
for i in tqdm(range(1000)):
    _ = get_predictions(many_examples,port=ports["restrserve"])

100%|██████████| 1000/1000 [00:36<00:00, 27.66it/s]


In [15]:
for i in tqdm(range(1000)):
    get_health(port=ports["restrserve"])

100%|██████████| 1000/1000 [00:09<00:00, 100.26it/s]


### FastAPI

In [16]:
get_predictions(example*2,port=ports["fastapi"]).json()

{'output': [0.0, 0.0]}

In [17]:
for i in tqdm(range(1000)):
    _ = get_predictions(example,port=ports["fastapi"])

100%|██████████| 1000/1000 [00:06<00:00, 162.92it/s]


In [18]:
for i in tqdm(range(1000)):
    _ = get_predictions(many_examples,port=ports["fastapi"])

100%|██████████| 1000/1000 [00:07<00:00, 127.62it/s]


In [19]:
for i in tqdm(range(1000)):
    get_health(port=ports["fastapi"])

100%|██████████| 1000/1000 [00:04<00:00, 226.09it/s]


# Keep Alive

In [12]:
instance=requests.Session()

### Plumber

In [21]:
for i in tqdm(range(1000)):
    _ = get_predictions(example, instance=instance, port=ports["plumber"])

100%|██████████| 1000/1000 [00:49<00:00, 20.21it/s]


In [22]:
for i in tqdm(range(1000)):
    _ = get_predictions(many_examples, instance=instance, port=ports["plumber"])

100%|██████████| 1000/1000 [00:52<00:00, 18.91it/s]


In [23]:
for i in tqdm(range(1000)):
    get_health(instance=instance, port=ports["plumber"])

100%|██████████| 1000/1000 [00:44<00:00, 22.27it/s]


### RestRserve

In [24]:
for i in tqdm(range(1000)):
    _ = get_predictions(example, instance=instance, port=ports["restrserve"])

100%|██████████| 1000/1000 [00:04<00:00, 207.69it/s]


In [25]:
for i in tqdm(range(1000)):
    _ = get_predictions(many_examples, instance=instance, port=ports["restrserve"])

100%|██████████| 1000/1000 [00:06<00:00, 155.17it/s]


In [26]:
for i in tqdm(range(1000)):
    get_health(instance=instance, port=ports["restrserve"])

100%|██████████| 1000/1000 [00:02<00:00, 421.02it/s]


### FastAPI

In [20]:
for i in tqdm(range(1000)):
    _ = get_predictions(example, instance=instance, port=ports["fastapi"])

100%|██████████| 1000/1000 [00:04<00:00, 215.08it/s]


In [21]:
for i in tqdm(range(1000)):
    _ = get_predictions(many_examples, instance=instance, port=ports["fastapi"])

100%|██████████| 1000/1000 [00:06<00:00, 164.84it/s]


In [22]:
for i in tqdm(range(1000)):
    get_health(instance=instance, port=ports["fastapi"])

100%|██████████| 1000/1000 [00:02<00:00, 345.38it/s]
